# Extract data from output files
### Analyze the output from a single LBANN run
March 9, 2020 \
April 6, 2020 : to store files in order of epochs \
April 21, 2020: added jupyter widgets to compare pixel intensity plots \
July 30, 2020: Perform analysis with stored histograms.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

## Extract folder

In [5]:
img_size=128
dict1={'scratch':'/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/{0}square/'.format(img_size),
    'proj':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
display(u)


interactive(children=(Select(description='x', options=('scratch', 'proj'), value='scratch'), Output()), _dom_c…

In [6]:
parent_dir=u.result
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20200911_083711_bsize64_spec_test_128_nospec', '2020…

In [16]:
result=w.result
main_dir=parent_dir+result
print(main_dir)

/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201202_200004_bsize256_scale0.5


In [17]:
# Get location for validation data
raw_data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/'
input_files_dict={'128':raw_data_dir+'128_square/dataset_2_smoothing_200k/norm_1_train_val.npy',
                 '512':raw_data_dir+'512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'}

In [18]:
# main_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/scaling_runs/20201031_205304_bsize512_scaling_2_8'

## Extract metrics info from log file

In [19]:
def f_extract_info(fname):
    '''
    Module to extract information from out.log files of Lbann training
    Reads in file name
    '''
    strg_lst=['objective','d_real','d_fake','gen','spec_loss','run time','mini-batch']
    keys=['training_'+strg for strg in strg_lst]
    dict1={}
    for category in ['training','validation']:
        for strg in strg_lst:
            try: 
                key=category+'_'+strg
                cmd='grep "{0}" {1} | grep "{2}"'.format(category,fname,strg)
        #         print(cmd)
                op1=sp.check_output(cmd,shell=True).decode('utf-8').split('\n')
                obj=np.array([strg.split(':')[-1] for strg in op1 if strg])
                dict1[key]=obj
            except Exception as e:
                print(e)
                dict1[key]=np.nan
    
    df=pd.DataFrame([])
    key_lst=['training_objective', 'training_d_real', 'training_d_fake', 'training_gen', 'training_spec_loss','validation_objective', 'validation_d_real', 'validation_d_fake', 'validation_gen','validation_spec_loss']
    col_list=['train_obj','train_dreal','train_dfake','train_gen','train_spec','val_obj','val_dreal','val_dfake','val_gen','val_spec']
    for col,key in zip(col_list,key_lst):
        try: 
            df[col]=dict1[key].astype(np.float)
        except: pass

    ### Need to remove the trailing 's' in the timings
    for col,key in zip(['train_time','val_time'],['training_run time','validation_run time']):
        df[col]=np.array([i[:-1] for i in dict1[key]]).astype(np.float)

    for col,key in zip(['train_batch_stats','val_batch_stats'],['training_mini-batch','validation_mini-batch']):
        df[col]=dict1[key]
        
    return df

def f_plot_metrics(df,col_list=['train_obj']):
    '''
    Plot multiple columns of the dataframe
    '''
    plt.figure()
    
    marker_lst=('o','*','H','D','.','x')
    marker=itertools.cycle(marker_lst)
    for col in col_list: plt.plot(df[col],linestyle='',marker=next(marker),label=col)
    plt.legend()
    plt.xlabel('Epoch')
    

In [20]:
strg=main_dir+'/out.log'
df_metrics=f_extract_info(glob.glob(strg)[0])

In [21]:
df_metrics.head()

,train_obj,train_dreal,train_dfake,train_gen,train_spec,val_obj,val_dreal,val_dfake,val_gen,val_spec,train_time,val_time,train_batch_stats,val_batch_stats
0,14.31850,1.101120,1.245270,5.91346,12.1174,12.43720,0.194797,0.068503,6.20195,11.9439,979.177,15.56330,"0.125896s mean, 4.54526s max, 0.026413s min, ...","0.158759s mean, 5.54897s max, 0.0201277s min,..."
1,10.67920,0.478576,0.454589,3.90584,11.6804,19.40340,0.193520,0.486256,12.83420,11.7787,978.630,8.37652,"0.123065s mean, 1.07305s max, 0.0260277s min,...","0.0854251s mean, 0.542614s max, 0.0200008s mi..."
2,10.53060,0.374815,0.337329,4.03607,11.5647,12.13950,0.385878,0.063189,5.86829,11.6442,939.058,9.13259,"0.118202s mean, 1.16204s max, 0.0250281s min,...","0.0931414s mean, 0.967496s max, 0.0158584s mi..."
3,10.20910,0.348556,0.334746,3.77005,11.5115,8.72345,0.131710,1.602550,1.27333,11.4317,983.972,10.09610,"0.126043s mean, 1.36s max, 0.0255064s min, 0....","0.102958s mean, 1.16817s max, 0.0159941s min,..."
4,9.66298,0.386800,0.381737,3.15317,11.4825,11.78650,0.114035,4.839210,1.11806,11.4305,1033.490,12.59080,"0.121025s mean, 0.744858s max, 0.0265892s min...","0.128429s mean, 1.70125s max, 0.0156333s min,..."


In [22]:
col_list=df_metrics.columns[~df_metrics.columns.str.endswith('stats')]
interact_manual(f_plot_metrics,col_list=SelectMultiple(options=col_list),df=fixed(df_metrics))


interactive(children=(SelectMultiple(description='col_list', options=('train_obj', 'train_dreal', 'train_dfake…

<function __main__.f_plot_metrics(df, col_list=['train_obj'])>

## Extract data from stored dataframe

In [65]:
### Load data
df=pd.read_pickle(main_dir+'/df_processed_unevenbins.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']


In [66]:
print(df.shape)
df.tail()


(3029, 22)


,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,...,chi_spec3,num_imgs,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_sdev,label
3024,17,15120,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.013539,0.014392,0.018271,0.046202,0.013123,5.427943,...,23.904068,256,1,0,"[1.053038279448139, 1.052594640982326, 0.26976...","[0.004301033001452868, 0.003405859369179162, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[89109.12003478035, 61121.490794439414, 42484....","[44790.62165222847, 21971.272168484316, 14620....",17-15120
3025,17,15125,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.006102,0.008910,0.004190,0.019203,0.003822,5.280135,...,22.821710,256,1,0,"[1.080944250331962, 1.0068393737685943, 0.2643...","[0.004089931158213837, 0.0034058493352592143, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[89251.00567082153, 62366.11761230043, 44170.6...","[45471.52471041433, 22101.327158119795, 14830....",17-15125
3026,17,15130,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.005667,0.006998,0.016968,0.029633,0.003779,5.087555,...,28.030732,256,0,0,"[1.156162510814215, 0.9170349254429082, 0.2402...","[0.003628146393992298, 0.003057008818143988, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[79539.11786871031, 56791.480681341156, 40634....","[40808.851626267475, 19152.351198834698, 13735...",17-15130
3027,17,15135,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.041674,0.010172,0.005276,0.057122,0.021431,4.808324,...,27.716370,256,0,0,"[1.0224920090584693, 1.060400003338137, 0.2895...","[0.0031028015291697892, 0.00259921688474107, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[81614.10638947133, 64140.02482197786, 46317.7...","[36699.65052661542, 22434.770251234593, 14658....",17-15135
3028,17,15140,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.011197,0.032792,0.011180,0.055169,0.010893,4.637206,...,29.153949,256,0,0,"[1.0604099956684585, 1.0438222752199304, 0.270...","[0.0035019363879523232, 0.0029312807201335813,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[74594.97208011337, 54434.59515774321, 40746.4...","[38801.64522138553, 19796.95668410786, 13777.2...",17-15140


In [67]:
df.columns

Index(['epoch', 'step', 'img_type', 'fname', 'chi_1a', 'chi_1b', 'chi_1c',
       'chi_1', 'chi_2', 'chi_imgvar', 'chi_spec1', 'chi_spec2', 'chi_spec3',
       'num_imgs', 'num_large', 'num_vlarge', 'hist_val', 'hist_err',
       'hist_bin_centers', 'spec_val', 'spec_sdev', 'label'],
      dtype='object')

### View best epochs

#### Locations with best chi_sqr

In [68]:
def f_get_best_chisqr_models(df,cutoff=0.2):
    '''
    Pick models with lowest 20% chi-square for multiple categories
    '''
    chi_sqr_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[chi_sqr_keys])
#     print(q_dict)
    
    df_sliced=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    return df_sliced

chi_sqr_keys=['epoch','step','chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
# index location of min/max values of chi squares
inds=[]
for key in ['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']:
    inds.append(df[key].idxmin(axis=1))
inds=np.array(inds)
df.loc[inds][chi_sqr_keys]


,epoch,step,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2
2071,11,10355,0.000190,0.029598,0.010777,0.040565,0.000358,4.979850,3.455367,353.032196
392,2,1960,0.357449,0.000715,0.005505,0.363669,0.344503,4.720601,8.303655,685.076985
1605,9,8025,0.047988,0.044115,0.000995,0.093098,0.013704,5.392463,1.612376,139.732948
2157,12,10785,0.005187,0.003908,0.003839,0.012934,0.002789,5.006427,0.817296,72.669877
1443,8,7215,0.001770,0.007261,0.005256,0.014287,0.000144,4.635581,0.710702,108.025095
27,0,135,1.708324,0.286004,0.013186,2.007514,1.133257,0.941963,2273.278844,50232.578964
2927,16,14635,0.022302,0.006281,0.008566,0.037149,0.016436,5.499386,0.166297,19.144361
2345,13,11725,0.006802,0.013845,0.006679,0.027327,0.000839,6.185571,0.214704,9.280948


In [69]:
### Plot chi-sqr values
# _=df_sliced.plot(x="step", y=["chi_1", "chi_imgvar", "chi_spec1"],style='.',marker='*')

In [70]:
### Plot number of very high pixel images
plt.figure()
plt.plot(df[df.img_type=='train_gen'].step,df[df.img_type=='train_gen'].num_vlarge,linestyle='',marker='*')
plt.xlabel('Steps in Epochs')
plt.ylabel('Number of very large pixel images from a batch of images')

# df[df.num_vlarge>10].tail()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Number of very large pixel images from a batch of images')

### View best steps

In [71]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [72]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [73]:
df_sliced=w.result
# df_sliced

### Pick best steps

In [97]:
best_step=[]
best_step.append(f_slice_df(df,cutoff=0.2,sort_col='chi_1',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.8,sort_col='chi_1b',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.2,sort_col='chi_spec1',head=2,display_flag=False).step.values)

best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, 17]
{'chi_1': 0.046516111566310483, 'chi_spec1': 0.7843071181810282}
[0, 17]
{'chi_1': 0.23415700442300208, 'chi_spec1': 4.612355500578111}
[0, 17]
{'chi_1': 0.046516111566310483, 'chi_spec1': 0.7843071181810282}
[ 3430  6460  7215  9610 11725 14635]


In [98]:
best_step=[3430,  6460,  7215,  9610, 11725, 14635, 9475]
# best_step=np.arange(40130,40135).astype(int)

In [99]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(7, 22)
[(3, 3430), (7, 6460), (8, 7215), (10, 9610), (13, 11725), (16, 14635), (10, 9475)]


In [100]:
df_best

,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,...,chi_spec3,num_imgs,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_sdev,label
686,3,3430,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.046158,0.000864,0.004573,0.051595,0.039143,5.302889,...,29.078899,256,0,0,"[1.2077936263243398, 0.7861442652533569, 0.278...","[0.004347834904205765, 0.0032818456269134415, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[102851.58780854754, 66322.90006638391, 52621....","[40079.00864060478, 18261.13817695571, 14765.7...",3-3430
1292,7,6460,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.064044,0.002865,0.032437,0.099346,0.060994,4.981114,...,31.085608,256,3,0,"[1.2602852557396946, 0.7601284688701946, 0.231...","[0.003018771620299291, 0.0025846803434975067, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[71227.00011204556, 57692.99514090234, 43794.2...","[27194.01608405591, 16755.517358011562, 11645....",7-6460
1443,8,7215,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.001770,0.007261,0.005256,0.014287,0.000144,4.635581,...,26.660547,256,0,0,"[1.1180239125121822, 0.9585238709909794, 0.260...","[0.0027643074054960306, 0.0022904287194995244,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[84329.10043138638, 66030.78595215855, 50091.2...","[40856.43584530588, 22058.066834169247, 15807....",8-7215
1895,10,9475,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.055284,0.016345,0.009287,0.080916,0.032991,5.697384,...,32.179091,256,1,0,"[1.2053368569708123, 0.8045394582028551, 0.250...","[0.0036129915393311642, 0.002485453816277658, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[118407.95097510237, 72547.68957355933, 46760....","[64082.397298141856, 26935.797974023113, 16378...",10-9475
1922,10,9610,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.003940,0.006625,0.004051,0.014616,0.001208,5.327169,...,30.157410,256,2,0,"[1.1337478935159708, 0.9323786362466882, 0.256...","[0.003335712626556639, 0.0024920608639151473, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[102162.56873700768, 71046.77774560265, 50834....","[57314.40280363705, 27108.872447341382, 16434....",10-9610
2345,13,11725,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.006802,0.013845,0.006679,0.027327,0.000839,6.185571,...,31.371691,256,2,0,"[1.1020187689476826, 0.9647219866473661, 0.269...","[0.005426296860002674, 0.004204742485739218, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[112965.17597660981, 73155.59742044513, 48361....","[60003.828938607694, 25726.59527676723, 16474....",13-11725
2927,16,14635,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.022302,0.006281,0.008566,0.037149,0.016436,5.499386,...,26.182687,256,1,0,"[1.0380869087530822, 1.0554727853701897, 0.282...","[0.004039959974189517, 0.00316288278275713, 0....","[-1.031746031584782, -0.6161616169043975, -0.3...","[87339.36366426758, 66288.28954601589, 45919.3...","[40269.57149342349, 26292.987872636368, 15181....",16-14635


In [101]:
df_best[col_list]

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
686,3-3430,0.051595,2.708747,29.078899,0.046158,0.000864,0.004573,0.039143,5.302889,228.700179,3,3430
1292,7-6460,0.099346,0.820235,31.085608,0.064044,0.002865,0.032437,0.060994,4.981114,85.347005,7,6460
1443,8-7215,0.014287,0.710702,26.660547,0.001770,0.007261,0.005256,0.000144,4.635581,108.025095,8,7215
1895,10-9475,0.080916,1.684029,32.179091,0.055284,0.016345,0.009287,0.032991,5.697384,156.797980,10,9475
1922,10-9610,0.014616,0.451900,30.157410,0.003940,0.006625,0.004051,0.001208,5.327169,48.804109,10,9610
2345,13-11725,0.027327,0.214704,31.371691,0.006802,0.013845,0.006679,0.000839,6.185571,9.280948,13,11725
2927,16-14635,0.037149,0.166297,26.182687,0.022302,0.006281,0.008566,0.016436,5.499386,19.144361,16,14635


## Plot pixel intensity and spectrum

In [102]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1


### Extract validation data
num_bkgnd=1000
fname=input_files_dict[str(img_size)]
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)

(1000, 128, 128)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [103]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    assert plot_type in ['hist','spec','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='grid':
            images=np.load(row.fname)[:,0,:,:]
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [104]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    
interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','grid']))

interactive(children=(SelectMultiple(description='labels_list', options=('3-3430', '7-6460', '8-7215', '10-947…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

In [105]:
df[(df.step>4400)& (df.step<5350)].plot(kind='line',x='step',y=['chi_1','chi_spec1'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='step'>

## View image block

In [ ]:
def f_get_img(df,step):
    df_temp=df[(df.step==step)]
    images=np.load(df_temp.fname.values[0])[:,0,:,:]
    return images

img_arr=f_get_img(df,8025)
# f_plot_grid(img_arr[:18],cols=6,fig_size=(10,5))

In [ ]:
f_plot_grid(s_val[100:118],cols=6,fig_size=(10,5))

### Plotting spread of spectrum of input data

In [ ]:

# plt.figure()

# idxs=np.random.randint(0,s_val.shape[0],size=50)
# arr=s_val[idxs]
# Pk = f_batch_spectrum(arr)
# k=np.arange(Pk.shape[1])
# for count,i in enumerate(Pk):
#     plt.plot(i)
    
# Pk = f_batch_spectrum(img_arr)
# mean,std = np.mean(Pk, axis=0),np.std(Pk, axis=0)/np.sqrt(Pk.shape[0])
# plt.plot(k, mean, 'k:',label='generated image')


# plt.legend()

# plt.ylabel(r'$P(k)$')
# plt.xlabel(r'$k$')
# plt.title('Power Spectrum: 50 input images')
# plt.yscale('log')



In [ ]:
fname='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201130_084039_bsize256_test_with_invtransform-scale0.5/gen_imgs_chkpt/epoch.12.step.11130_20201201_072154/dump_outs/trainer0/model0/sgd.testing.epoch.0.step.0_gen_img_instance1_activation_output0.npy'
np.load(fname).shape
